# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
df_path = "output_data/cities.csv"
cities_data = pd.read_csv(df_path)
cities_data

,Unnamed: 0,City,City Lat,City Long,Max Temp,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime
0,0,Bluff,-46.6000,168.3333,53.01,88,100,4.00,NZ,1618367277
1,1,Port Elizabeth,-33.9180,25.5701,66.20,94,90,10.36,ZA,1618367278
2,2,Rikitea,-23.1203,-134.9692,77.70,66,7,6.82,PF,1618367278
3,3,Zhigansk,66.7697,123.3711,19.54,87,100,21.00,RU,1618367279
4,4,Bandarbeyla,9.4942,50.8122,78.49,75,72,5.41,SO,1618367279
...,...,...,...,...,...,...,...,...,...,...
577,577,Hobyo,5.3505,48.5268,80.96,66,99,8.30,SO,1618367575
578,578,Clifton,40.8584,-74.1638,57.99,62,90,2.46,US,1618367298
579,579,Otjiwarongo,-20.4637,16.6477,60.31,41,0,15.37,NaN,1618367576
580,580,Béziers,43.3476,3.2190,39.20,70,0,6.91,FR,1618367576


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [34]:
#Access maps with my unique API key
gmaps.configure(api_key=g_key)

#Use Latitude and Longitude from weather dataframe as locations
locations = cities_data[["City Lat", "City Long"]]

#Use humidity from dataframe as the weight
humidity = cities_data["Humidity"]


In [36]:
#create heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)

#Can also set parameters to max intensity as per weights for humidity and point radius settings like this below
#heatmap_layer.max_intensity = max(humidity)
#heatmap_layer.point_radius = 3

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
#generate a new dataframe consiting of cities that have ideal weather by filtering cities_data by desirable weather attributes
ideal_destinations_df = cities_data.loc[(cities_data["Max Temp"] > 70) & 
                               (cities_data["Max Temp"] < 80) & 
                               (cities_data["Wind Speed"] < 10) & 
                               (cities_data["Cloud Coverage"] == 0) &
                               (cities_data["Humidity"] < 75), :].reset_index(drop=True).dropna()
ideal_destinations_df

,Unnamed: 0,City,City Lat,City Long,Max Temp,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime
0,82,Salalah,17.0151,54.0924,77.00,73,0,4.61,OM,1618367300
1,96,São Filipe,14.8961,-24.4956,76.62,60,0,9.46,CV,1618367321
2,106,Saint-Philippe,-21.3585,55.7679,78.80,73,0,6.91,RE,1618367326
3,159,Byron Bay,-28.6500,153.6167,75.20,43,0,9.22,AU,1618367290
4,166,Kaffrine,14.1059,-15.5508,76.21,36,0,7.09,SN,1618367359
7,351,Jaciara,-15.9653,-54.9683,70.14,72,0,2.73,BR,1618367426
8,470,Saint-Pierre,-21.3393,55.4781,78.80,73,0,6.91,RE,1618367520
9,546,Barra de Tecoanapa,16.5167,-98.7500,78.06,72,0,5.84,MX,1618367559


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
#Create a dataframe to store acceptable city hotels in 
hotel_df = ideal_destinations_df.loc[:,["City", "City Country", "City Lat", "City Long"]]

#add column with header name "Hotel"
hotel_df["Hotel"] = ""
hotel_df

,City,City Country,City Lat,City Long,Hotel
0,Salalah,OM,17.0151,54.0924,
1,São Filipe,CV,14.8961,-24.4956,
2,Saint-Philippe,RE,-21.3585,55.7679,
3,Byron Bay,AU,-28.6500,153.6167,
4,Kaffrine,SN,14.1059,-15.5508,
7,Jaciara,BR,-15.9653,-54.9683,
8,Saint-Pierre,RE,-21.3393,55.4781,
9,Barra de Tecoanapa,MX,16.5167,-98.7500,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
